In [86]:
%pip install jaro-winkler

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [87]:
%pip install textdistance

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [88]:
%pip install pyphonetics

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 235 kB 2.9 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [89]:
%pip install wordfreq

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 56.8 MB 29 kB/s  eta 0:00:01
     |████████████████████████████████| 273 kB 67.9 MB/s 
     |████████████████████████████████| 173 kB 89.4 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.5.1-py3-none-any.whl size=56830992 sha256=18841c11c7bdadd5d3a585cf14393bd03a5533a9ecfe98fc3779a2be8a877136
  Stored in directory: /tmp/xdg_cache/pip/wheels/d8/d4/2d/741daa49d00231b4c5afbec15e79f361d9e76961f1a5fc02e5
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=ce9fedd4ae8ae8ac60e8830cdb0898477f0b9594fe8d88bd0c91d542e7b1a9f6
  Stored in directory: /tmp/xdg_cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
  Created wheel for langcodes: filename=langcodes-3.2.1-py3-none-any.whl size=169379 sha256=ed4b0c57c5921789f8e1bf37218425357b21f734cc95c54ec2d3ffb8222b9a5d
  S

In [95]:
%pip install cyhunspell

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 3.8 MB 2.9 MB/s eta 0:00:02
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [133]:
import nltk
import numpy as np

from jaro import jaro_winkler_metric
from textdistance import hamming, levenshtein
from pyphonetics import RefinedSoundex
from wordfreq import word_frequency
from hunspell import Hunspell
from nltk.tokenize import word_tokenize

In [134]:
nltk.download('punkt', quiet=True)

True

In [135]:
def sort_candidates(word, candidates):
    if len(candidates) == 0:
            return []
    
    soundex = RefinedSoundex()
    features = np.array([[hamming(word, candidate),
                          levenshtein(word, candidate),
                          soundex.distance(word, candidate, metric='levenshtein'),
                          1 - jaro_winkler_metric(word, candidate),
                          1 - word_frequency(candidate, 'en')] for candidate in candidates])
    w = ((features - features.min(axis=0) + 1e-9) / (features.max(axis=0) - features.min(axis=0) + 1e-9)).mean(axis=1)
    return list(np.array(candidates)[np.argsort(w)])

In [136]:
def check(text):
    hunspell = Hunspell('en_US')
    words = [word for word in word_tokenize(text, 'english') if word.isalpha()]
    words = list(filter(lambda w: not hunspell.spell(w), words))
    
    #top 10 candidates
    for word in words:
        print(f'Strange word >{word}<, possible fixes: {sort_candidates(word, hunspell.suggest(word))[:10]}')

In [137]:
check("Inside This Machine is the secand studioo album from My Passion. The album was recorded in April 2010 at Outhouse Studios in Reading, England and was releazed on 18 April 2011. The band released the track Seven Birds as a free download from their website in July 2010.")

Strange word >secand<, possible fixes: ['second', 'secant', 'sec and', 'sec-and']
Strange word >studioo<, possible fixes: ['studio', 'studio o', 'studios', 'studied']
Strange word >releazed<, possible fixes: ['released', 'realized', 'relearned']
